In [12]:
import pyodbc, time, os
import pandas as pd

import urllib
from sqlalchemy import create_engine, types
import fesh_creds

In [ ]:
files = [line for line in os.listdir() if line.endswith('xlsx')]
len(files)

In [ ]:
#Checker
checker = ['id',  'text', 'source', 'in_reply_to_status_id', 'create_date', 'create_time', 'followers',
           'fav_count', 'statuses_count', 'loacation', 'verified', 'name', 'screen_name']

In [ ]:
#Assemble all files
dfs = []
for item in files:
    df = pd.read_excel(item)
    if df.columns.tolist() == checker:
        dfs.append(df)
        print(f"{item} done")
    else:
        print(f"Inconsistent column names in {item}")

In [ ]:
combo = pd.concat(dfs).reset_index(drop=True)
combo.shape

In [ ]:
[line for line in combo.columns]

In [ ]:
#Rename id to tweet_id
combo = combo.rename(columns = {'id':'tweet_id'})
combo.keys()

In [ ]:
dtypes = ['bigint NOT NULL',
 'varchar (254)',
 'varchar (254)',
 'varchar (254)',
 'varchar (254) NOT NULL',
 'varchar (254) NOT NULL',
 'bigint',
 'bigint',
 'bigint',
 'varchar (254)',
 'varchar (7)',
 'varchar (254)',
 'varchar (254)',
         ]

In [ ]:
for i, j in zip(combo.columns.tolist(), dtypes):
    print(f"{i}  {'>'*10} {j}")

In [ ]:
tweets = [line for line in combo.text if not line.startswith('RT')]
retweets = [line for line in combo.text if line.startswith('RT')]

print(len(tweets), len(retweets)

In [ ]:
#Some RTS have no content after the starting RT
oya = []
for line in retweets:
    x = line.split()
    try:
        oya.append(x[1])
    except:
        continue

In [ ]:
len(retweets) - len(oya)

In [ ]:
from collections import Counter

In [ ]:
Counter(oya).most_common(20)

# Onboarding

In [ ]:
#Make connection
params = urllib.parse.quote_plus(
    'DRIVER={ODBC Driver 17 for SQL Server};SERVER=FESH\FESHQL; DATABASE=feshdb; Trusted_Connection=yes'
)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [10]:
import os
os.getcwd()

'C:\\Users\\USER\\Desktop\\2NYP\\2NYP Database'

In [9]:
fesh_creds.server

'FESH\\FESHQL'

In [14]:
################### BUILD CXN STRING ###################
#Create connection
server = fesh_creds.server
database = fesh_creds.database
username = fesh_creds.username
password = fesh_creds.password
trust = 'yes'
port = '1433'
cxn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER='+server+';PORT='+port+';DATABASE='+database+';'
    )

#Connect
cursor = cxn.cursor()

print("Connected to the Server... Cursor hot!")
time.sleep(2)

Connected to the Server... Cursor hot!


In [ ]:
#query
query = "DROP TABLE IF EXISTS _2NYP; CREATE TABLE _2NYP(tweet_id VARCHAR(50) NOT NULL);"

#create table
cursor.execute(query)
print("Created _2NYP table")
time.sleep(2)

#create the rest of the columns
for d, t in zip(combo.columns.tolist()[1:], dtypes[1:]):
    query2 = "ALTER TABLE " + '_2NYP' + " ADD " + d + " " + t
    cursor.execute(query2)

#commit
cxn.commit()

#Cleanup
cursor.close()
cxn.close()
print("fConnection closed.")

In [ ]:
#Write data to LifeRenewals table and replace if exists
try:
    combo.to_sql('_2NYP', engine, if_exists='append', index=False)
    print("Successfully writtent to warehouse")
except:
    print("Something bad occurred!")

In [ ]:
#Read data from DW
query = "SELECT * from _2NYP;"
dfx = pd.read_sql(query, cxn)

In [ ]:
combo.fav_count.max()